<a href="https://colab.research.google.com/github/MontyRex/ML-labs/blob/main/%D0%A2%D0%B5%D1%81%D1%82_%D0%BE%D1%87%D0%B8%D1%81%D1%82%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PATH_TO_FILE: str = '/content/diamond.csv' # Загрузка набора данных
## p.s. в левой панеле нажать на папку и перетащить туда файл,
## а wince.csv заменить на свой вариант (ПКМ -> копировать путь)

df = pd.read_csv(PATH_TO_FILE) # функция превращение данных из CSV в DataFrame

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

Здесь я пытался объединить все таблицы в один дф.

---

Поправить, если будет галлюцинировать

In [4]:
def load_and_validate_tables() -> Dict[str, pd.DataFrame]:
    """Загрузка и валидация всех 8 таблиц"""

    tables = {}

    # Загрузка всех таблиц
    table_files = {
        'cities': 'cities.xlsx',
        'districts': 'districts.xlsx',
        'streets': 'streets.xlsx',
        'buildings': 'buildings.xlsx',
        'blackouts': 'blackouts.xlsx',
        'blackouts_buildings': 'blackouts_buildings.xlsx',
        'folk_districts': 'folk_districts.xlsx',
        'big_folk_districts': 'big_folk_districts.xlsx'
    }

    for table_name, file_name in table_files.items():
        try:
            tables[table_name] = pd.read_excel(file_name)
            print(f"✅ {table_name}: {tables[table_name].shape[0]} строк, {tables[table_name].shape[1]} столбцов")
        except Exception as e:
            print(f"❌ Ошибка загрузки {table_name}: {e}")
            return None

    # Проверка первичных ключей на уникальность
    print("\n=== ПРОВЕРКА ПЕРВИЧНЫХ КЛЮЧЕЙ ===")
    primary_keys = {
        'cities': 'id',
        'districts': 'id',
        'streets': 'id',
        'buildings': 'id',
        'blackouts': 'id',
        'folk_districts': 'id',
        'big_folk_districts': 'id'
    }

    for table, pk in primary_keys.items():
        if table in tables and pk in tables[table].columns:
            unique_count = tables[table][pk].nunique()
            total_count = len(tables[table])
            status = "✅ OK" if unique_count == total_count else "❌ ДУБЛИКАТЫ"
            print(f"{table}.{pk}: {unique_count}/{total_count} {status}")

    return tables

In [5]:
print("\n1. 📥 ЗАГРУЗКА ДАННЫХ")
tables = load_and_validate_tables()
if tables is None:
    raise Exception("Не удалось загрузить данные")


1. 📥 ЗАГРУЗКА ДАННЫХ
✅ cities: 2 строк, 2 столбцов
✅ districts: 5 строк, 2 столбцов
✅ streets: 2023 строк, 3 столбцов
✅ buildings: 58199 строк, 10 столбцов
✅ blackouts: 25264 строк, 7 столбцов
✅ blackouts_buildings: 1048575 строк, 2 столбцов
✅ folk_districts: 3594 строк, 2 столбцов
✅ big_folk_districts: 45 строк, 2 столбцов

=== ПРОВЕРКА ПЕРВИЧНЫХ КЛЮЧЕЙ ===
cities.id: 2/2 ✅ OK
districts.id: 5/5 ✅ OK
streets.id: 2023/2023 ✅ OK
buildings.id: 58199/58199 ✅ OK
blackouts.id: 25264/25264 ✅ OK
folk_districts.id: 3594/3594 ✅ OK
big_folk_districts.id: 45/45 ✅ OK


Анализ качества данны


In [6]:
def comprehensive_quality_analysis(tables: Dict[str, pd.DataFrame]):
    """Комплексный анализ качества данных для всех таблиц"""

    print("=== КОМПЛЕКСНЫЙ АНАЛИЗ КАЧЕСТВА ДАННЫХ ===\n")

    quality_report = {}

    for table_name, df in tables.items():
        print(f"📊 {table_name.upper()}: {df.shape[0]} строк, {df.shape[1]} столбцов")

        # Анализ пропущенных значений
        missing_data = df.isnull().sum()
        missing_pct = (missing_data / len(df)) * 100

        if missing_data.sum() > 0:
            print("   Пропущенные значения:")
            for col, count in missing_data[missing_data > 0].items():
                print(f"     {col}: {count} ({missing_pct[col]:.1f}%)")
        else:
            print("   ✅ Пропущенных значений нет")

        # Анализ дубликатов
        duplicates = df.duplicated().sum()
        if duplicates > 0:
            print(f"   ❌ Дубликаты: {duplicates} строк")
        else:
            print("   ✅ Дубликатов нет")

        # Анализ аномальных значений для числовых колонок
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            print("   Числовые колонки:")
            for col in numeric_cols:
                if col != 'id':  # Пропускаем ID
                    q1 = df[col].quantile(0.25)
                    q3 = df[col].quantile(0.75)
                    iqr = q3 - q1
                    lower_bound = q1 - 1.5 * iqr
                    upper_bound = q3 + 1.5 * iqr

                    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
                    if len(outliers) > 0:
                        print(f"     {col}: {len(outliers)} выбросов")

        # Анализ текстовых данных
        text_cols = df.select_dtypes(include=['object']).columns
        if len(text_cols) > 0:
            print("   Текстовые колонки:")
            for col in text_cols:
                unique_count = df[col].nunique()
                sample_values = df[col].dropna().unique()[:3]
                print(f"     {col}: {unique_count} уникальных значений, примеры: {sample_values}")

        print("   ---")

        quality_report[table_name] = {
            'missing_values': missing_data[missing_data > 0].to_dict(),
            'duplicates': duplicates,
            'numeric_columns': list(numeric_cols),
            'text_columns': list(text_cols)
        }

    return quality_report

In [7]:
# 2. Анализ качества
print("\n2. АНАЛИЗ КАЧЕСТВА ДАННЫХ")
quality_report = comprehensive_quality_analysis(tables)


2. 🔍 АНАЛИЗ КАЧЕСТВА ДАННЫХ
=== КОМПЛЕКСНЫЙ АНАЛИЗ КАЧЕСТВА ДАННЫХ ===

📊 CITIES: 2 строк, 2 столбцов
   ✅ Пропущенных значений нет
   ✅ Дубликатов нет
   Текстовые колонки:
     id: 2 уникальных значений, примеры: ['24aa25303f0f91ede8ee0c0d324c8ee3' 'b428b92bb123994a56234bb6eeeed414']
     name: 2 уникальных значений, примеры: ['Артем' 'Владивосток']
   ---
📊 DISTRICTS: 5 строк, 2 столбцов
   ✅ Пропущенных значений нет
   ✅ Дубликатов нет
   Текстовые колонки:
     id: 5 уникальных значений, примеры: ['b428b92bb123994a56234bb6eeeed414' '24aa25303f0f91ede8ee0c0d324c8ee3'
 '504f6b8bd5daeba64beb62bda8b3c12a']
     name: 5 уникальных значений, примеры: ['Ленинский район' 'Первомайский район' 'Первореченский район']
   ---
📊 STREETS: 2023 строк, 3 столбцов
   ✅ Пропущенных значений нет
   ✅ Дубликатов нет
   Текстовые колонки:
     id: 2023 уникальных значений, примеры: ['ceba6f5d616341f44f5fcae0425d020b' 'a7fc0a136f4b1e18a8365d3d29c65c05'
 'd3ef58b013fa6d05ad5b6cf4210e0fc7']
     name: 1

А сейчас будем чистить нулики, преобразовывать данные и делать прочие непотребства муа-ха-ха


In [8]:
def clean_all_tables(tables: Dict[str, pd.DataFrame]):
    """Комплексная очистка и преобразование всех таблиц"""

    cleaned_tables = {}

    # 1. CITIES - очистка таблицы городов
    print("🧹 Очистка таблицы CITIES...")
    cities = tables['cities'].copy()
    # Удаляем дубликаты по id и name
    cities = cities.drop_duplicates(subset=['id'], keep='first')
    cities = cities.drop_duplicates(subset=['name'], keep='first')
    cities['name'] = cities['name'].str.strip().str.title()
    cleaned_tables['cities'] = cities
    print(f"   Результат: {len(cities)} записей")

    # 2. DISTRICTS - очистка таблицы районов
    print("🧹 Очистка таблицы DISTRICTS...")
    districts = tables['districts'].copy()
    districts = districts.drop_duplicates(subset=['id'], keep='first')
    districts['name'] = districts['name'].str.strip().str.title()
    cleaned_tables['districts'] = districts
    print(f"   Результат: {len(districts)} записей")

    # 3. STREETS - очистка таблицы улиц
    print("🧹 Очистка таблицы STREETS...")
    streets = tables['streets'].copy()
    streets = streets.drop_duplicates(subset=['id'], keep='first')
    streets['name'] = streets['name'].str.strip()

    # Проверяем существование city_id в таблице cities
    if 'city_id' in streets.columns and 'cities' in cleaned_tables:
        valid_city_ids = set(cleaned_tables['cities']['id'])
        streets = streets[streets['city_id'].isin(valid_city_ids)]

    cleaned_tables['streets'] = streets
    print(f"   Результат: {len(streets)} записей")

    # 4. FOLK_DISTRICTS и BIG_FOLK_DISTRICTS
    print("🧹 Очистка таблиц FOLK_DISTRICTS...")
    folk_districts = tables['folk_districts'].copy()
    folk_districts = folk_districts.drop_duplicates(subset=['id'], keep='first')
    folk_districts['name'] = folk_districts['name'].str.strip().str.title()
    cleaned_tables['folk_districts'] = folk_districts

    big_folk_districts = tables['big_folk_districts'].copy()
    big_folk_districts = big_folk_districts.drop_duplicates(subset=['id'], keep='first')
    big_folk_districts['name'] = big_folk_districts['name'].str.strip().str.title()
    cleaned_tables['big_folk_districts'] = big_folk_districts
    print(f"   Результат: {len(folk_districts)} folk, {len(big_folk_districts)} big folk")

    # 5. BUILDINGS - комплексная очистка таблицы зданий
    print("🧹 Очистка таблицы BUILDINGS...")
    buildings = tables['buildings'].copy()
    initial_count = len(buildings)

    # Удаляем дубликаты
    buildings = buildings.drop_duplicates(subset=['id'], keep='first')

    # Обрабатываем пропуски
    if 'is_fake' in buildings.columns:
        buildings['is_fake'] = buildings['is_fake'].fillna(False)

    # Стандартизируем номера домов
    if 'number' in buildings.columns:
        buildings['number'] = buildings['number'].astype(str).str.strip()

    # Проверяем внешние ключи
    if 'street_id' in buildings.columns and 'streets' in cleaned_tables:
        valid_street_ids = set(cleaned_tables['streets']['id'])
        buildings = buildings[buildings['street_id'].isin(valid_street_ids)]

    if 'district_id' in buildings.columns and 'districts' in cleaned_tables:
        valid_district_ids = set(cleaned_tables['districts']['id'])
        buildings = buildings[buildings['district_id'].isin(valid_district_ids)]

    if 'city_id' in buildings.columns and 'cities' in cleaned_tables:
        valid_city_ids = set(cleaned_tables['cities']['id'])
        buildings = buildings[buildings['city_id'].isin(valid_city_ids)]

    print(f"   Результат: {initial_count} → {len(buildings)} записей")
    cleaned_tables['buildings'] = buildings

    # 6. BLACKOUTS - тщательная очистка таблицы отключений
    print("🧹 Очистка таблицы BLACKOUTS...")
    blackouts = tables['blackouts'].copy()
    initial_count = len(blackouts)

    # Удаляем дубликаты
    blackouts = blackouts.drop_duplicates(subset=['id'], keep='first')

    # Преобразование дат с обработкой ошибок
    date_columns = ['start_date', 'end_date']
    for col in date_columns:
        if col in blackouts.columns:
            blackouts[col] = pd.to_datetime(blackouts[col], errors='coerce')

    # Удаляем записи с некорректными датами
    blackouts = blackouts.dropna(subset=['start_date'])

    # Удаляем отключения, где end_date раньше start_date
    if 'end_date' in blackouts.columns:
        valid_dates = blackouts['end_date'] >= blackouts['start_date']
        blackouts = blackouts[valid_dates | blackouts['end_date'].isna()]

    # Очистка текстовых полей
    text_columns = ['description', 'type', 'initiator_name', 'source']
    for col in text_columns:
        if col in blackouts.columns:
            blackouts[col] = blackouts[col].astype(str).str.strip()
            # Замена пустых строк на "Не указано"
            blackouts[col] = blackouts[col].replace('nan', 'Не указано')

    # Стандартизация типов отключений
    if 'type' in blackouts.columns:
        type_mapping = {
            'electricity': 'электричество',
            'heat': 'отопление',
            'hot_water': 'горячая вода',
            'cold_water': 'холодная вода',
        }
        blackouts['type'] = blackouts['type'].str.lower().replace(type_mapping)

    print(f"   Результат: {initial_count} → {len(blackouts)} записей")
    cleaned_tables['blackouts'] = blackouts

    # 7. BLACKOUTS_BUILDINGS - очистка таблицы связей
    print("🧹 Очистка таблицы BLACKOUTS_BUILDINGS...")
    blackouts_buildings = tables['blackouts_buildings'].copy()
    initial_count = len(blackouts_buildings)

    # Удаляем дубликаты
    blackouts_buildings = blackouts_buildings.drop_duplicates()

    # Проверяем внешние ключи
    valid_blackout_ids = set(cleaned_tables['blackouts']['id'])
    valid_building_ids = set(cleaned_tables['buildings']['id'])

    blackouts_buildings = blackouts_buildings[
        blackouts_buildings['blackout_id'].isin(valid_blackout_ids) &
        blackouts_buildings['building_id'].isin(valid_building_ids)
    ]

    print(f"   Результат: {initial_count} → {len(blackouts_buildings)} записей")
    cleaned_tables['blackouts_buildings'] = blackouts_buildings

    return cleaned_tables

In [9]:
print("\n3. 🧹 ОЧИСТКА ВСЕХ ТАБЛИЦ")
cleaned_tables = clean_all_tables(tables)


3. 🧹 ОЧИСТКА ВСЕХ ТАБЛИЦ
🧹 Очистка таблицы CITIES...
   Результат: 2 записей
🧹 Очистка таблицы DISTRICTS...
   Результат: 5 записей
🧹 Очистка таблицы STREETS...
   Результат: 2023 записей
🧹 Очистка таблиц FOLK_DISTRICTS...
   Результат: 3594 folk, 45 big folk
🧹 Очистка таблицы BUILDINGS...
   Результат: 58199 → 34606 записей
🧹 Очистка таблицы BLACKOUTS...
   Результат: 25264 → 25264 записей
🧹 Очистка таблицы BLACKOUTS_BUILDINGS...
   Результат: 1048575 → 175590 записей


Проверим целостность таблиц


In [10]:
def validate_data_integrity(cleaned_tables: Dict[str, pd.DataFrame]):
    """Проверка целостности данных после очистки"""

    print("\n=== ПРОВЕРКА ЦЕЛОСТНОСТИ ДАННЫХ ===")

    # 1. Проверка внешних ключей
    print("🔗 Проверка внешних ключей:")

    # buildings.street_id → streets.id
    if 'buildings' in cleaned_tables and 'streets' in cleaned_tables:
        valid_streets = set(cleaned_tables['streets']['id'])
        buildings_streets = set(cleaned_tables['buildings']['street_id'].dropna())
        invalid_streets = buildings_streets - valid_streets
        print(f"   buildings.street_id → streets.id: {len(invalid_streets)} невалидных ссылок")

    # blackouts_buildings.blackout_id → blackouts.id
    if 'blackouts_buildings' in cleaned_tables and 'blackouts' in cleaned_tables:
        valid_blackouts = set(cleaned_tables['blackouts']['id'])
        bb_blackouts = set(cleaned_tables['blackouts_buildings']['blackout_id'])
        invalid_blackouts = bb_blackouts - valid_blackouts
        print(f"   blackouts_buildings.blackout_id → blackouts.id: {len(invalid_blackouts)} невалидных ссылок")

    # 2. Проверка согласованности данных
    print("\n📊 Проверка согласованности данных:")

    # Количество отключений vs связей
    if 'blackouts' in cleaned_tables and 'blackouts_buildings' in cleaned_tables:
        blackouts_with_buildings = cleaned_tables['blackouts_buildings']['blackout_id'].nunique()
        total_blackouts = len(cleaned_tables['blackouts'])
        coverage = (blackouts_with_buildings / total_blackouts) * 100
        print(f"   Отключения с привязкой к зданиям: {blackouts_with_buildings}/{total_blackouts} ({coverage:.1f}%)")

    # Статистика по зданиям
    if 'buildings' in cleaned_tables and 'blackouts_buildings' in cleaned_tables:
        buildings_with_blackouts = cleaned_tables['blackouts_buildings']['building_id'].nunique()
        total_buildings = len(cleaned_tables['buildings'])
        affected_pct = (buildings_with_blackouts / total_buildings) * 100
        print(f"   Зданий с отключениями: {buildings_with_blackouts}/{total_buildings} ({affected_pct:.1f}%)")

In [11]:
print("\n4. 🔗 ПРОВЕРКА ЦЕЛОСТНОСТИ")
validate_data_integrity(cleaned_tables)


4. 🔗 ПРОВЕРКА ЦЕЛОСТНОСТИ

=== ПРОВЕРКА ЦЕЛОСТНОСТИ ДАННЫХ ===
🔗 Проверка внешних ключей:
   buildings.street_id → streets.id: 0 невалидных ссылок
   blackouts_buildings.blackout_id → blackouts.id: 0 невалидных ссылок

📊 Проверка согласованности данных:
   Отключения с привязкой к зданиям: 23517/25264 (93.1%)
   Зданий с отключениями: 21159/34606 (61.1%)


Теперь можно и норманый датасет сделать

In [12]:
def create_comprehensive_dataset(cleaned_tables: Dict[str, pd.DataFrame]):
    """Создание комплексного аналитического датасета"""

    print("\n=== СОЗДАНИЕ АНАЛИТИЧЕСКОГО ДАТАСЕТА ===")

    # Начинаем с таблицы отключений
    analytical_data = cleaned_tables['blackouts'].copy()

    # Присоединяем информацию о зданиях через таблицу связей
    analytical_data = analytical_data.merge(
        cleaned_tables['blackouts_buildings'],
        left_on='id',
        right_on='blackout_id',
        how='left',
        suffixes=('', '_bb')
    )

    # Присоединяем информацию о зданиях
    analytical_data = analytical_data.merge(
        cleaned_tables['buildings'],
        left_on='building_id',
        right_on='id',
        how='left',
        suffixes=('', '_building')
    )

    # Присоединяем информацию об улицах
    analytical_data = analytical_data.merge(
        cleaned_tables['streets'],
        left_on='street_id',
        right_on='id',
        how='left',
        suffixes=('', '_street')
    )

    # Присоединяем информацию о районах
    analytical_data = analytical_data.merge(
        cleaned_tables['districts'],
        left_on='district_id',
        right_on='id',
        how='left',
        suffixes=('', '_district')
    )

    # Присоединяем информацию о народных районах
    if 'folk_district_id' in analytical_data.columns:
        analytical_data = analytical_data.merge(
            cleaned_tables['folk_districts'],
            left_on='folk_district_id',
            right_on='id',
            how='left',
            suffixes=('', '_folk')
        )

    # Присоединяем информацию о больших народных районах
    if 'big_folk_district_id' in analytical_data.columns:
        analytical_data = analytical_data.merge(
            cleaned_tables['big_folk_districts'],
            left_on='big_folk_district_id',
            right_on='id',
            how='left',
            suffixes=('', '_big_folk')
        )

    print(f"✅ Создан объединенный датасет: {len(analytical_data)} записей")
    print(f"   Столбцы: {list(analytical_data.columns)}")

    return analytical_data

In [13]:
print("\n5. 📊 СОЗДАНИЕ АНАЛИТИЧЕСКОГО ДАТАСЕТА")
analytical_data = create_comprehensive_dataset(cleaned_tables)


5. 📊 СОЗДАНИЕ АНАЛИТИЧЕСКОГО ДАТАСЕТА

=== СОЗДАНИЕ АНАЛИТИЧЕСКОГО ДАТАСЕТА ===
✅ Создан объединенный датасет: 177337 записей
   Столбцы: ['id', 'start_date', 'end_date', 'description', 'type', 'initiator_name', 'source', 'blackout_id', 'building_id', 'id_building', 'street_id', 'number', 'district_id', 'is_fake', 'folk_district_id', 'big_folk_district_id', 'type_building', 'city_id', 'coordinates', 'id_street', 'name', 'city_id_street', 'id_district', 'name_district', 'id_folk', 'name_folk', 'id_big_folk', 'name_big_folk']


Получилось что-то более-менее вразумительное

In [14]:
def create_advanced_features(analytical_data: pd.DataFrame):
    """Создание расширенных признаков для анализа"""

    print("\n=== СОЗДАНИЕ РАСШИРЕННЫХ ПРИЗНАКОВ ===")

    df = analytical_data.copy()
    new_features_count = 0

    # ВРЕМЕННЫЕ ПРИЗНАКИ
    if 'start_date' in df.columns:
        # Базовая длительность
        df['duration_hours'] = (df['end_date'] - df['start_date']).dt.total_seconds() / 3600

        # Временные компоненты
        df['start_hour'] = df['start_date'].dt.hour
        df['start_day'] = df['start_date'].dt.day
        df['start_month'] = df['start_date'].dt.month
        df['start_year'] = df['start_date'].dt.year
        df['day_of_week'] = df['start_date'].dt.day_name()
        df['day_of_week_num'] = df['start_date'].dt.dayofweek
        df['is_weekend'] = df['start_date'].dt.dayofweek >= 5
        df['season'] = (df['start_date'].dt.month % 12 + 3) // 3

        # Категории времени суток
        conditions = [
            (df['start_hour'] >= 0) & (df['start_hour'] < 6),
            (df['start_hour'] >= 6) & (df['start_hour'] < 12),
            (df['start_hour'] >= 12) & (df['start_hour'] < 18),
            (df['start_hour'] >= 18) & (df['start_hour'] < 24)
        ]
        choices = ['Ночь', 'Утро', 'День', 'Вечер']
        df['time_of_day'] = np.select(conditions, choices, default='Неизвестно')

        # Категории длительности
        df['duration_category'] = pd.cut(
            df['duration_hours'],
            bins=[0, 1, 6, 24, 168, float('inf')],
            labels=['<1ч', '1-6ч', '6-24ч', '1-7д', '>7д'],
            right=False
        )

        new_features_count += 12

    # ГЕОГРАФИЧЕСКИЕ ПРИЗНАКИ
    if 'name_district' in df.columns:
        # Флаг для центральных районов (пример)
        central_districts = ['центральный', 'ленинский', 'советский']  # нужно адаптировать под реальные данные
        df['is_central_district'] = df['name_district'].str.lower().isin([d.lower() for d in central_districts])
        new_features_count += 1

    # ПРИЗНАКИ ТИПОВ ОТКЛЮЧЕНИЙ
    if 'type' in df.columns:
        # Бинарные признаки для основных типов
        df['is_electricity'] = df['type'].str.contains('электр|свет', case=False, na=False)
        df['is_water'] = df['type'].str.contains('вод', case=False, na=False)
        df['is_heating'] = df['type'].str.contains('отопл|теп', case=False, na=False)
        df['is_gas'] = df['type'].str.contains('газ', case=False, na=False)

        new_features_count += 4

    # ПРИЗНАКИ ИНИЦИАТОРОВ
    if 'initiator_name' in df.columns:
        # Категоризация инициаторов
        emergency_keywords = ['авария', 'аварий', 'поломка', 'breakdown', 'emergency']
        planned_keywords = ['планов', 'ремонт', 'обслуживание', 'planned', 'maintenance']

        df['is_emergency'] = df['initiator_name'].str.lower().str.contains('|'.join(emergency_keywords), na=False)
        df['is_planned'] = df['initiator_name'].str.lower().str.contains('|'.join(planned_keywords), na=False)

        new_features_count += 2

    # ПРИЗНАКИ ЗДАНИЙ
    if 'type_building' in df.columns:
        # Категоризация типов зданий
        residential_keywords = ['жилой', 'дом', 'квартир', 'residential']
        commercial_keywords = ['коммерч', 'офис', 'магазин', 'commercial']

        df['is_residential'] = df['type_building'].str.lower().str.contains('|'.join(residential_keywords), na=False)
        df['is_commercial'] = df['type_building'].str.lower().str.contains('|'.join(commercial_keywords), na=False)

        new_features_count += 2

    print(f"✅ Создано {new_features_count} новых признаков")
    print(f"   Итоговый размер: {df.shape[1]} столбцов")

    return df

In [15]:
print("\n6. 🎯 СОЗДАНИЕ ПРИЗНАКОВ")
final_data = create_advanced_features(analytical_data)

print(f"\n🎉 ПОЛНАЯ ПОДГОТОВКА ЗАВЕРШЕНА!")
print(f"📈 Итоговый датасет: {final_data.shape[0]:,} строк, {final_data.shape[1]} столбцов")
print(f"📅 Диапазон дат: {final_data['start_date'].min()} - {final_data['start_date'].max()}")
print(f"🏢 Уникальных зданий: {final_data['building_id'].nunique():,}")
print(f"📍 Уникальных районов: {final_data['name_district'].nunique() if 'name_district' in final_data.columns else 'N/A'}")

# Сохранение подготовленных данных
final_data.to_csv('prepared_blackouts_data.csv', index=False, encoding='utf-8')
print("💾 Данные сохранены в prepared_blackouts_data.csv")


6. 🎯 СОЗДАНИЕ ПРИЗНАКОВ

=== СОЗДАНИЕ РАСШИРЕННЫХ ПРИЗНАКОВ ===
✅ Создано 21 новых признаков
   Итоговый размер: 48 столбцов

🎉 ПОЛНАЯ ПОДГОТОВКА ЗАВЕРШЕНА!
📈 Итоговый датасет: 177,337 строк, 48 столбцов
📅 Диапазон дат: 2018-01-01 00:08:00 - 2019-12-31 20:29:00
🏢 Уникальных зданий: 21,159
📍 Уникальных районов: 5
💾 Данные сохранены в prepared_blackouts_data.csv
